In [ ]:
!git clone https://github.com/SKfaizan-786/EAST-text-detection.git
%cd EAST-text-detection

In [ ]:
# !pip install -r requirements.txt

In [ ]:
import os
import cv2
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import scale
import matplotlib.pyplot as plt
from glob import glob

# Base paths inside the cloned repo
DATA_DIR = "data/icdar2015"
TRAIN_IMG_DIR = f"{DATA_DIR}/train_images"
TRAIN_LABEL_DIR = f"{DATA_DIR}/train_labels"
TRAIN_MAP_DIR = f"{DATA_DIR}/train_maps"
os.makedirs(TRAIN_MAP_DIR, exist_ok=True)

TEST_IMG_DIR = f"{DATA_DIR}/test_images"
TEST_LABEL_DIR = f"{DATA_DIR}/test_labels"
TEST_MAP_DIR = f"{DATA_DIR}/test_maps"
os.makedirs(TEST_MAP_DIR, exist_ok=True)

print("Train images:", len(glob(f"{TRAIN_IMG_DIR}/*.jpg")))
print("Train labels:", len(glob(f"{TRAIN_LABEL_DIR}/*.txt")))

In [ ]:
def parse_label_file(label_path):
    """
    Parses an ICDAR2015 label txt file and returns a list of quads.
    Each quad is [(x1,y1),(x2,y2),(x3,y3),(x4,y4)].
    """
    quads = []
    with open(label_path, "r", encoding="utf-8-sig") as f:
        for line in f.readlines():
            parts = line.strip().split(',')
            if len(parts) >= 8:
                coords = list(map(float, parts[:8]))
                quad = [(coords[i], coords[i+1]) for i in range(0, 8, 2)]
                quads.append(quad)
    return quads

def canonicalize_quad(quad):
    """
    Return a consistent clockwise ordering of quad vertices starting
    from the top-left (approx).
    quad: list of 4 (x,y) tuples
    """
    pts = np.array(quad, dtype=np.float32)
    cx, cy = np.mean(pts, axis=0)
    angles = np.arctan2(pts[:,1] - cy, pts[:,0] - cx)
    order = np.argsort(angles)
    pts = pts[order]
    sums = pts[:,0] + pts[:,1]
    start = np.argmin(sums)
    pts = np.roll(pts, -start, axis=0)
    return pts.tolist()

def shortest_edge_length(quad):
    q = np.array(quad, dtype=np.float32)
    lengths = []
    for i in range(4):
        p1 = q[i]
        p2 = q[(i+1)%4]
        lengths.append(np.linalg.norm(p1 - p2))
    return float(min(lengths))

def shrink_quad(quad, ratio=0.3):
    """
    Shrink quadrilateral inward using shapely.
    ratio ~0.3 recommended in EAST paper.
    """
    poly = Polygon(quad)
    if not poly.is_valid:
        return poly
    return scale(poly, xfact=1-ratio, yfact=1-ratio, origin='centroid')

In [ ]:
def generate_maps_with_nmap(img, quads, shrink_ratio=0.3):
    """
    Returns score_map, geo_map, n_map.
    n_map is same HxW single channel: N_Q* value for text pixels, 0 outside.
    """
    H, W = img.shape[:2]
    score_map = np.zeros((H, W), dtype=np.uint8)
    geo_map   = np.zeros((H, W, 8), dtype=np.float32)
    n_map     = np.zeros((H, W), dtype=np.float32)

    for quad in quads:
        quad = canonicalize_quad(quad)
        
        min_edge = shortest_edge_length(quad)
        N_q = 4.0 * min_edge

        shrunk_poly = shrink_quad(quad, shrink_ratio)
        if shrunk_poly.is_empty:
            continue

        pts = np.array(list(shrunk_poly.exterior.coords)[:-1], dtype=np.int32)
        mask = np.zeros((H, W), dtype=np.uint8)
        cv2.fillPoly(mask, [pts], 1)
        ys, xs = np.where(mask == 1)

        score_map[ys, xs] = 1
        n_map[ys, xs] = N_q

        quad_arr = np.array(quad, dtype=np.float32)
        for y, x in zip(ys, xs):
            for k in range(4):
                dx = quad_arr[k,0] - x
                dy = quad_arr[k,1] - y
                geo_map[y,x,2*k]   = dx
                geo_map[y,x,2*k+1] = dy

    return score_map, geo_map, n_map

In [ ]:
sample_imgs = sorted(glob(f"{TRAIN_IMG_DIR}/*.jpg"))[:5]
for img_path in sample_imgs:
    name = os.path.splitext(os.path.basename(img_path))[0]
    label_path = os.path.join(TRAIN_LABEL_DIR, f"gt_{name}.txt")
    img = cv2.imread(img_path)
    quads = parse_label_file(label_path)
    score_map, geo_map, n_map = generate_maps_with_nmap(img, quads)

    np.save(os.path.join(TRAIN_MAP_DIR, f"{name}_score.npy"), score_map)
    np.save(os.path.join(TRAIN_MAP_DIR, f"{name}_geo.npy"), geo_map)
    np.save(os.path.join(TRAIN_MAP_DIR, f"{name}_nmap.npy"), n_map)

print("✅ Saved 5 sample maps to train_maps/")

In [ ]:
# visualize one example
idx = 0
img_path = sample_imgs[idx]
name = os.path.splitext(os.path.basename(img_path))[0]
img = cv2.imread(img_path)[:,:,::-1]  # BGR->RGB
score_map = np.load(os.path.join(TRAIN_MAP_DIR, f"{name}_score.npy"))
nmap = np.load(os.path.join(TRAIN_MAP_DIR, f"{name}_nmap.npy"))

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
ax1.imshow(img); ax1.set_title('Image'); ax1.axis('off')
ax2.imshow(score_map, cmap='gray'); ax2.set_title('Score Map'); ax2.axis('off')
ax3.imshow(nmap, cmap='jet'); ax3.set_title('Normalization Map'); ax3.axis('off')
plt.show()

In [ ]:
def process_split(img_dir, label_dir, out_dir):
    img_paths = sorted(glob(f"{img_dir}/*.jpg"))
    for img_path in img_paths:
        name = os.path.splitext(os.path.basename(img_path))[0]
        label_path = os.path.join(label_dir, f"gt_{name}.txt")
        if not os.path.exists(label_path):
            continue
        img = cv2.imread(img_path)
        quads = parse_label_file(label_path)
        score_map, geo_map, n_map = generate_maps_with_nmap(img, quads)
        np.save(os.path.join(out_dir, f"{name}_score.npy"), score_map)
        np.save(os.path.join(out_dir, f"{name}_geo.npy"), geo_map)
        np.save(os.path.join(out_dir, f"{name}_nmap.npy"), n_map)

# Process training and test sets
process_split(TRAIN_IMG_DIR, TRAIN_LABEL_DIR, TRAIN_MAP_DIR)
process_split(TEST_IMG_DIR, TEST_LABEL_DIR, TEST_MAP_DIR)
print("✅ Finished generating score, geometry, and normalization maps for all images.")